In [1]:
import os
import pandas as pd
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
from tqdm import tqdm
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import clipboard
import base64
from io import BytesIO
from PIL import Image
from proxyattention.meta_utils import read_pickle, return_grouped_results


In [12]:
# !python result_aggregator.py

100%|███████████████████████████████████████████| 19/19 [00:10<00:00,  1.88it/s]


In [2]:
combined_df = read_pickle(fname = "./results/aggregated_runs.csv")[0]


In [3]:
combined_df.shape

(611, 39)

# Clean if required

In [4]:
broken_runs = list(combined_df[combined_df["global_run_count"].isna() == True]["index"].values)
broken_runs[:2]

['/media/eragon/data/CODE/thesis_runs/proper_runs/hypers_01052023_192308/events.out.tfevents.1682962474.pop-os',
 '/media/eragon/data/CODE/thesis_runs/proper_runs/proxy_run_28042023_165052/events.out.tfevents.1682693458.pop-os']

In [5]:
combined_df = combined_df[combined_df["global_run_count"].isna() == False]
combined_df["global_run_count"] = combined_df["global_run_count"].astype(int)
combined_df["global_run_count"].unique()

array([ 5,  6, 15, 16, 25, 26, 35, 36, 40, 20, 21,  1,  8])

In [6]:
broken_runs.extend(combined_df.query('global_run_count < 10')["index"].values)
len(broken_runs)

135

In [7]:
# Delete broken runs
[Path.unlink(Path(x)) for x in broken_runs]

FileNotFoundError: [Errno 2] No such file or directory: '/media/eragon/data/CODE/thesis_runs/proper_runs/hypers_01052023_192308/events.out.tfevents.1682962474.pop-os'

In [103]:
def dir_empty(path):
    empty = True
    for item in path.glob('*'):
        if item.is_file():
            empty = False
        if item.is_dir() and not dir_empty(item):
            empty = False
    # if empty:
        # path.rmdir()  # Remove if you just want to have the result
    return empty

# dir_empty([x for x in Path("./results").glob("*") if x.is_dir()])
# [Path.rmdir(x) for x in Path("./results").glob("*") if x.is_dir() and dir_empty(x)]
[Path.rmdir(x) for x in Path("./runs").glob("*") if x.is_dir() and dir_empty(x)]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

# Continue here

In [8]:
combined_df.head()

,index,proxy_step,Loss/Train,Acc/Train,global_run_count,Loss/Val,Acc/Val,experiment_name,image_size,batch_size,...,name_fn,writer,label_map,rev_label_map,num_classes,dataset_sizes,criterion,save_path,final_acc,Number_Chosen
0,/media/eragon/data/CODE/thesis_runs/proper_run...,False,0.004473,94.371521,5,0.008572,83.978638,hypers,224,64,...,<function get_parent_name at 0x7fd559fca680>,<tensorboardX.writer.SummaryWriter object at 0...,"{0: 'n02085620-Chihuahua', 1: 'n02085936-Malte...","{'n02085620-Chihuahua': 0, 'n02085936-Maltese_...",51,"{'train': 4495, 'val': 4494}",CrossEntropyLoss(),/media/eragon/data/CODE/thesis_runs/proper_run...,83.97863818424567,NaN
1,/media/eragon/data/CODE/thesis_runs/proper_run...,True,0.006403,89.454948,6,0.004788,91.522026,hypers,224,64,...,<function get_parent_name at 0x7fd559fca680>,<tensorboardX.writer.SummaryWriter object at 0...,"{0: 'n02085620-Chihuahua', 1: 'n02085936-Malte...","{'n02085620-Chihuahua': 0, 'n02085936-Maltese_...",51,"{'train': 4495, 'val': 4494}",CrossEntropyLoss(),/media/eragon/data/CODE/thesis_runs/proper_run...,91.52202937249666,380.0
2,/media/eragon/data/CODE/thesis_runs/proper_run...,False,0.000564,99.666298,15,0.00361,93.257675,hypers,224,64,...,<function get_parent_name at 0x7fd559fca680>,<tensorboardX.writer.SummaryWriter object at 0...,"{0: 'n02085620-Chihuahua', 1: 'n02085936-Malte...","{'n02085620-Chihuahua': 0, 'n02085936-Maltese_...",51,"{'train': 4495, 'val': 4494}",CrossEntropyLoss(),/media/eragon/data/CODE/thesis_runs/proper_run...,93.25767690253672,NaN
3,/media/eragon/data/CODE/thesis_runs/proper_run...,True,0.004289,92.593384,16,0.003292,93.979507,hypers,224,64,...,<function get_parent_name at 0x7fd559fca680>,<tensorboardX.writer.SummaryWriter object at 0...,"{0: 'n02085620-Chihuahua', 1: 'n02085936-Malte...","{'n02085620-Chihuahua': 0, 'n02085936-Maltese_...",51,"{'train': 4685, 'val': 4684}",CrossEntropyLoss(),/media/eragon/data/CODE/thesis_runs/proper_run...,93.97950469684031,278.0
4,/media/eragon/data/CODE/thesis_runs/proper_run...,False,0.000234,99.866516,25,0.001474,97.374275,hypers,224,64,...,<function get_parent_name at 0x7fd559fca680>,<tensorboardX.writer.SummaryWriter object at 0...,"{0: 'n02085620-Chihuahua', 1: 'n02085936-Malte...","{'n02085620-Chihuahua': 0, 'n02085936-Maltese_...",51,"{'train': 4495, 'val': 4494}",CrossEntropyLoss(),/media/eragon/data/CODE/thesis_runs/proper_run...,97.37427681352915,NaN


In [9]:
combined_df.shape

(595, 39)

In [11]:
combined_df[combined_df["global_run_count"]!= "0"]["proxy_steps"].value_counts()

[5, 'p', 9, 'p', 9, 'p', 9, 'p', 4]    521
[20, 'p', 19]                           53
[40]                                    19
Name: proxy_steps, dtype: int64

In [24]:
combined_df["experiment_name"].value_counts()

hypers       358
proxy_run    192
transform     43
0              2
Name: experiment_name, dtype: int64

# Image code

In [21]:
# filtered_df.iloc[1].original_images

In [22]:
# filtered_df.iloc[1].converted_proxy

# Preprocess

In [12]:
def check_proxy(string): return "p" in str(string)

In [13]:
def calc_stats(values):
    return f"min: {values.min()} \nmax: {values.max()} \navg: {values.mean()}"

In [14]:
def convert_float(df, cols, totype= float):
    for col in cols:
        df[col] = df[col].astype(totype)

In [15]:
combined_df = combined_df.fillna(0)
# col to check for proxy
# combined_df["has_proxy"] = combined_df["proxy_steps"].apply(check_proxy)
# Fix naming
combined_df = combined_df.rename(columns={"Acc/Val":"accuracy", "proxy_steps":"step_schedule"})
# Fix types
convert_float(combined_df, ["change_subset_attention", "proxy_threshold", "accuracy"], float)
convert_float(combined_df, ["global_run_count"], int)
convert_float(combined_df, ["transfer_imagenet"], bool)

# ignore failed runs
# combined_df = combined_df[combined_df["global_run_count"]!=0]


In [16]:
combined_df["has_proxy"] = combined_df["step_schedule"].apply(check_proxy)

In [17]:
combined_df.loc[combined_df["has_proxy"] == False, ["pixel_replacement_method", "gradient_method"]] = "baseline"

# Grouped Results

In [18]:
# def return_grouped_results(df, group_cols ,filter = None, index_cols = (["ds_name", ("accuracy")]), print_latex = False):
#     if filter != None:
#         df = df.reset_index()
#         for key in filter.keys():
#             df = df[df[key] == filter[key]]
#     final_df = pd.DataFrame(df.groupby(group_cols, as_index=True).mean(numeric_only = True)["accuracy"]).sort_values(index_cols, ascending=False)
#     if print_latex == True:
#         clipboard.copy(final_df.to_latex())

#     return final_df



In [30]:

return_grouped_results(combined_df, ["ds_name", "pixel_replacement_method", "transfer_imagenet", "gradient_method", "step_schedule", "change_subset_attention", "proxy_threshold", "model", "proxy_image_weight"], filter={"experiment_name": "transform"})

accuracy
ds_name pixel_replacement_method transfer_imagenet gradient_method step_schedule                       change_subset_attention proxy_threshold model                proxy_image_weight           
dogs    blended                  True              gradcamplusplus [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] 0.8                     0.80            vit_base_patch16_224 0.1                 86.055481
                                                                                                                               0.40            vit_base_patch16_224 0.1                 84.646194
                                                                                                                               0.10            vit_base_patch16_224 0.1                 83.396380
                                                                   [20, 'p', 19]                       0.8                     0.80            vit_base_patch16_224 0.1                 77.851952
                                                                                                                               0.10            vit_base_patch16_224 0.1                 77.555257
                                                                                                                               0.40            vit_base_patch16_224 0.1                 76.939624
        baseline                 True              baseline        [40]                                0.8                     0.10            vit_base_patch16_224 0.1                 71.562080
                                                                                                                               0.85            vit_base_patch16_224 0.1                 66.777924
                                                                                                                               0.80            vit_base_patch16_224 0.1                 65.242546
                                                                                                                               0.40            vit_base_patch16_224 0.1                 60.703159

In [31]:
return_grouped_results(combined_df, ["ds_name","model", "gradient_method", "step_schedule", "change_subset_attention"])

accuracy
ds_name      model                gradient_method step_schedule                       change_subset_attention           
plantdisease resnet18             gradcamplusplus [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] 0.80                     99.147778
             efficientnet_b0      gradcamplusplus [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] 0.80                     99.101112
             resnet50             gradcamplusplus [20, 'p', 19]                       0.80                     99.023333
             resnet18             baseline        [40]                                0.80                     98.949997
             resnet50             baseline        [40]                                0.80                     98.949997
             efficientnet_b0      baseline        [40]                                0.80                     98.900002
             resnet18             gradcamplusplus [20, 'p', 19]                       0.80                     98.826668
             resnet50             gradcamplusplus [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] 0.80                     98.786666
             efficientnet_b0      gradcamplusplus [20, 'p', 19]                       0.80                     98.680000
dogs         resnet50             gradcamplusplus [20, 'p', 19]                       0.80                     95.497701
                                                  [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] 0.80                     95.495351
             efficientnet_b0      gradcamplusplus [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] 0.20                     94.672288
                                                                                      0.80                     93.442382
                                                                                      0.95                     93.339572
             resnet50             baseline        [40]                                0.80                     92.011574
             efficientnet_b0      gradcamplusplus [20, 'p', 19]                       0.80                     91.410772
             resnet18             gradcamplusplus [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] 0.80                     87.411429
                                  gradcam         [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] 0.80                     86.384286
             efficientnet_b0      baseline        [40]                                0.80                     85.825546
             vit_base_patch16_224 gradcamplusplus [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] 0.80                     84.569055
             resnet18             gradcam         [20, 'p', 19]                       0.80                     80.473335
             vit_base_patch16_224 gradcamplusplus [20, 'p', 19]                       0.80                     77.448944
             resnet18             baseline        [40]                                0.80                     71.414997
             vit_base_patch16_224 baseline        [40]                                0.80                     66.071427
cifar100     resnet50             gradcamplusplus [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] 0.80                     83.106666
                                                  [20, 'p', 19]                       0.80                     82.090001
             efficientnet_b0      gradcamplusplus [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] 0.80                     81.665000
                                                  [20, 'p', 19]                       0.80                     79.037500
             resnet50             baseline        [40]                                0.80                     78.699997
             resnet18             gradcamplusplus [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] 0.80                     76.778000
                                                  [20, 'p', 19]                       0.80                     75.559998
             efficientnet_b0      baseline        [40]                                0.80                     75.22000

In [43]:
return_grouped_results(combined_df, ["ds_name", "transfer_imagenet", "gradient_method", "step_schedule", "change_subset_attention", "proxy_threshold", "model", "proxy_image_weight"], filter = { "ds_name":"plantdisease"})

accuracy
ds_name      transfer_imagenet gradient_method step_schedule                       change_subset_attention proxy_threshold model           proxy_image_weight           
plantdisease True              gradcamplusplus [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] 0.8                     0.85            resnet18        0.2                 99.147778
                                                                                                                           efficientnet_b0 0.2                 99.101112
                                               [20, 'p', 19]                       0.8                     0.85            resnet50        0.2                 99.023333
                               baseline        [40]                                0.8                     0.85            resnet18        0.2                 98.949997
                                                                                                                           resnet50        0.2                 98.949997
                                                                                                                           efficientnet_b0 0.2                 98.900002
                               gradcamplusplus [20, 'p', 19]                       0.8                     0.85            resnet18        0.2                 98.826668
                                               [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] 0.8                     0.85            resnet50        0.2                 98.786666
                                               [20, 'p', 19]                       0.8                     0.85            efficientnet_b0 0.2                 98.680000

In [44]:
return_grouped_results(combined_df, ["ds_name", "transfer_imagenet", "gradient_method", "step_schedule", "change_subset_attention", "proxy_threshold", "model", "proxy_image_weight"], filter = {"model":"efficientnet_b0"})

accuracy
ds_name      transfer_imagenet gradient_method step_schedule                       change_subset_attention proxy_threshold model           proxy_image_weight           
plantdisease True              gradcamplusplus [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] 0.80                    0.85            efficientnet_b0 0.2                 99.101112
                               baseline        [40]                                0.80                    0.85            efficientnet_b0 0.2                 98.900002
                               gradcamplusplus [20, 'p', 19]                       0.80                    0.85            efficientnet_b0 0.2                 98.680000
dogs         True              gradcamplusplus [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] 0.20                    0.80            efficientnet_b0 0.8                 94.985541
                                                                                                           0.10            efficientnet_b0 0.1                 94.911736
                                                                                                           0.40            efficientnet_b0 0.95                94.890021
                                                                                                                                           0.1                 94.845623
                                                                                                           0.10            efficientnet_b0 0.8                 94.786770
                                                                                                           0.40            efficientnet_b0 0.8                 94.782388
                                                                                                           0.85            efficientnet_b0 0.4                 94.762321
                                                                                                           0.10            efficientnet_b0 0.4                 94.737558
                                                                                                           0.80            efficientnet_b0 0.1                 94.723004
                                                                                                           0.85            efficientnet_b0 0.1                 94.658141
                                                                                                                                           0.8                 94.612412
                                                                                                           0.10            efficientnet_b0 0.95                94.565847
                                                                                                           0.80            efficientnet_b0 0.4                 94.493133
                                                                                                           0.40            efficientnet_b0 0.4                 94.491127
                                                                                                           0.80            efficientnet_b0 0.95                94.342921
                                                                                   0.80                    0.10            efficientnet_b0 0.1                 94.259889
                                                                                   0.20                    0.85            efficientnet_b0 0.95                94.168065
                                                                                   0.80                    0.40            efficientnet_b0 0.1                 94.059174
                                                                                                           0.80            efficientnet_b0 0.1                 93.934564
                                                                                                        

In [45]:
pd.set_option('display.max_rows', 500)
return_grouped_results(combined_df, ["ds_name", "pixel_replacement_method", "transfer_imagenet", "gradient_method", "step_schedule", "change_subset_attention", "proxy_threshold", "model", "proxy_image_weight"])

accuracy
ds_name      pixel_replacement_method transfer_imagenet gradient_method step_schedule                       change_subset_attention proxy_threshold model                proxy_image_weight           
plantdisease blended                  True              gradcamplusplus [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] 0.80                    0.85            resnet18             0.2                 99.147778
                                                                                                                                                    efficientnet_b0      0.2                 99.101112
                                                                        [20, 'p', 19]                       0.80                    0.85            resnet50             0.2                 99.023333
             baseline                 True              baseline        [40]                                0.80                    0.85            resnet18             0.2                 98.949997
                                                                                                                                                    resnet50             0.2                 98.949997
                                                                                                                                                    efficientnet_b0      0.2                 98.900002
             blended                  True              gradcamplusplus [20, 'p', 19]                       0.80                    0.85            resnet18             0.2                 98.826668
                                                                        [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] 0.80                    0.85            resnet50             0.2                 98.786666
                                                                        [20, 'p', 19]                       0.80                    0.85            efficientnet_b0      0.2                 98.680000
dogs         blended                  True              gradcamplusplus [20, 'p', 19]                       0.80                    0.85            resnet50             0.2                 95.497701
                                                                        [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] 0.80                    0.85            resnet50             0.2                 95.495351
                                                                                                            0.20                    0.80            efficientnet_b0      0.8                 94.985541
                                                                                                                                    0.10            efficientnet_b0      0.1                 94.911736
                                                                                                                                    0.40            efficientnet_b0      0.95                94.890021
                                                                                                                                                                         0.1                 94.845623
                                                                                                                                    0.10            efficientnet_b0      0.8                 94.786770
                                                                                                                                    0.40            efficientnet_b0      0.8                 94.782388
                                                                                                                                    0.85            efficientnet_b0      0.4                 94.762321
                                                                                                                                    0.10            efficientnet_b0      0.4                 94.737558
           

In [46]:

return_grouped_results(combined_df, ["ds_name", "pixel_replacement_method", "transfer_imagenet", "gradient_method", "step_schedule", "change_subset_attention", "proxy_threshold"], filter = { "model":"efficientnet_b0", "has_proxy":True,})

accuracy
ds_name      pixel_replacement_method transfer_imagenet gradient_method step_schedule                       change_subset_attention proxy_threshold           
plantdisease blended                  True              gradcamplusplus [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] 0.80                    0.85             99.101112
                                                                        [20, 'p', 19]                       0.80                    0.85             98.680000
dogs         blended                  True              gradcamplusplus [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] 0.20                    0.40             94.752290
                                                                                                                                    0.10             94.750478
                                                                                                                                    0.80             94.636150
                                                                                                                                    0.85             94.550235
                                                                                                            0.95                    0.10             93.690255
                                                                                                            0.80                    0.85             93.499807
                                                                                                                                    0.10             93.437621
                                                                                                            0.95                    0.40             93.428386
                                                                                                            0.80                    0.80             93.428107
                                                                                                                                    0.40             93.389638
                                                                                                            0.95                    0.80             93.363819
                                                                                                                                    0.85             92.817858
                                                                        [20, 'p', 19]                       0.80                    0.85             91.410772
cifar100     blended                  True              gradcamplusplus [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] 0.80                    0.85             81.665000
                                                                        [20, 'p', 19]                       0.80                    0.85             79.037500
caltech101   blended                  True              gradcamplusplus [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] 0.80                    0.85             97.773901
                                                                        [20, 'p', 19]                       0.80                    0.85             96.697289
asl          blended                  True              gradcamplusplus [20, 'p', 19]                       0.80                    0.85             99.903330
                                                                        [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] 0.80                    0.85             99.844443

In [47]:
return_grouped_results(combined_df, ["ds_name", "pixel_replacement_method", "transfer_imagenet", "gradient_method", "step_schedule", "model"])

accuracy
ds_name      pixel_replacement_method transfer_imagenet gradient_method step_schedule                       model                          
plantdisease blended                  True              gradcamplusplus [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] resnet18              99.147778
                                                                                                            efficientnet_b0       99.101112
                                                                        [20, 'p', 19]                       resnet50              99.023333
             baseline                 True              baseline        [40]                                resnet18              98.949997
                                                                                                            resnet50              98.949997
                                                                                                            efficientnet_b0       98.900002
             blended                  True              gradcamplusplus [20, 'p', 19]                       resnet18              98.826668
                                                                        [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] resnet50              98.786666
                                                                        [20, 'p', 19]                       efficientnet_b0       98.680000
dogs         blended                  True              gradcamplusplus [20, 'p', 19]                       resnet50              95.497701
                                                                        [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] resnet50              95.495351
                                                                                                            efficientnet_b0       93.905352
             baseline                 True              baseline        [40]                                resnet50              92.011574
             blended                  True              gradcamplusplus [20, 'p', 19]                       efficientnet_b0       91.410772
                                                                        [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] resnet18              87.411429
                                                        gradcam         [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] resnet18              86.384286
             baseline                 True              baseline        [40]                                efficientnet_b0       85.825546
             blended                  True              gradcamplusplus [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] vit_base_patch16_224  84.569055
                                                        gradcam         [20, 'p', 19]                       resnet18              80.473335
                                                        gradcamplusplus [20, 'p', 19]                       vit_base_patch16_224  77.448944
             baseline                 True              baseline        [40]                                resnet18              71.414997
                                                                                                            vit_base_patch16_224  66.071427
cifar100     blended                  True              gradcamplusplus [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] resnet50              83.106666
                                                                        [20, 'p', 19]                       resnet50              82.090001
                                                                        [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] efficientnet_b0       81.665000
                                                                        [20, 'p', 19]                       efficientnet_b0       79.037500
             baseline                 True              baseline        [40]                                resnet50              78.699997
             blended                  True              gradcamplusplus

In [49]:
return_grouped_results(combined_df, ["ds_name", "pixel_replacement_method", "transfer_imagenet", "gradient_method", "change_subset_attention", "step_schedule", "proxy_image_weight",  "model", "global_run_count"])

accuracy
ds_name      pixel_replacement_method transfer_imagenet gradient_method change_subset_attention step_schedule                       proxy_image_weight model                global_run_count            
plantdisease blended                  True              gradcamplusplus 0.80                    [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] 0.2                resnet50             40                 99.820000
                                                                                                                                                       efficientnet_b0      35                 99.730003
                                                                                                                                                       resnet18             40                 99.650002
                                                                                                                                                                            25                 99.580002
                                                                                                                                                                            35                 99.519997
                                                                                                                                                       efficientnet_b0      25                 99.510002
                                                                                                                                                                            15                 99.500000
                                                                                                                                                                            40                 99.449997
                                                                                                                                                       resnet18             15                 99.360001
                                                                                                                                                                            26                 99.339996
                                                                                                                                                       resnet50             25                 99.339996
                                                                                                                                                                            35                 99.330002
                                                                                                [20, 'p', 19]                       0.2                resnet18             40                 99.309998
                                                                                                [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] 0.2                efficientnet_b0      36                 99.239998
                                                                                                [20, 'p', 19]                       0.2                resnet50             40                 99.180000
                                                                                                [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] 0.2                resnet50             26                 99.139999
                                                                                                [20, 'p', 19]                       0.2                resnet50             21                 99.089996
                                                                                                [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] 0.2                efficientnet_b0      16                 99.040001
                                                                                                [20, 'p', 19]                       0.2                efficientnet_b0      